In [ ]:
from importlib import reload
import pymongo
import gridfs
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from scipy.sparse import csc_matrix, csr_matrix
import pickle
import pretty_midi
import sys
import copy
from collections import namedtuple
import timeit
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
# import modules, including a reload statement so that they can be reimported after a change to the methods 
import src.midi_utils as midi_utils
reload(midi_utils)

import src.data as data
reload(data)

import src.models as models
reload(models)

import src.ml_classes as ml_classes
reload(ml_classes)

### Predicting Velocities
Getting predictions out of a few models

In [ ]:
for i in os.scandir('training_data/mtt'):
    print(i.is_dir())

In [ ]:
model_datas, seconds = data.folder2examples('training_data/midi_val_8', sparse=False, use_base_key=True, beats_per_ex=16, sub_beats=2)
model_datas_pred = copy.deepcopy(model_datas)

In [ ]:
rands, idx = data.n_rand_examples(model_datas)
rands, idx = data.n_rand_examples(model_datas_s)

### Simple Bi LSTM Model: Zero vs Non-Zero Entries
Here was the first time I realised the MSE_zero + MSE_note issue might pose a problem. 

In [ ]:
model_input_reqs, model_output_reqs = models.get_model_reqs(['H', 'V_mean'], ['V'])

hidden_state = 256
lstm_layers = 2
dense_layers = 1
dense_size = 128
seq_model = models.create_simple_LSTM_RNN(model_input_reqs, model_output_reqs, seq_length=seq_length, dense_layers=dense_layers, dense_size=dense_size)
seq_model.summary()
tf.keras.utils.plot_model(seq_model)

# load some weights
seq_model.load_weights('89-0.31.hdf5')

#### Predict

In [ ]:
V_pred = seq_model.predict({md.name + '_in': md.data for md in model_datas.values()})

# get predictions for indices where a note exists, or doesn't exist
V_pred_ones = V_pred[np.where(model_datas['H'].data == 1)]
V_pred_zeros = V_pred[np.where(model_datas['H'].data != 1)]

#### Some plotting of note velocity predictions: mostly zeros or ones

In [ ]:
# plot frequencies of different velocities - we see that the model is biased towards predicting values close to 0 or 1
plt.figure(figsize=(9,6))
plt.yscale('log')
plt.hist(V_pred.flatten(), bins=40)
plt.title('Model 004 Velocities')
plt.xlabel('Velocity')
plt.ylabel('Frequency')
# plt.savefig('004-velocities')

In [ ]:
# how about plotting velocities for the positions where there are notes?
# all the zero velocity predictions are gone. I.e. this model is just learning to reproduce input.
plt.figure(figsize=(9,6))
# plt.yscale('log')
plt.hist(V_pred_ones, bins=40)
plt.title('Model 004 Velocities')
plt.xlabel('Velocity')
plt.ylabel('Frequency')

In [ ]:
# velocities for positions where there are no notes
plt.figure(figsize=(9,6))
plt.yscale('log')
plt.hist(V_pred_zeros, bins=40)
plt.title('Model 004 Velocities')
plt.xlabel('Velocity')
plt.ylabel('Frequency')

#### look at correlation between real and predicted velocities

In [ ]:
np.corrcoef(V_pred_ones, model_datas['V'].data[np.where(model_datas['H'].data == 1)])
# V_pred_ones.shape
# model_datas['V'].data[np.where(model_datas['H'].data == 1)]

In [ ]:
plt.scatter(V_pred_ones, model_datas['V'].data[np.where(model_datas['H'].data == 1)], alpha=0.03)

#### write predictions to midi

### Simple Autoencoder Prediction

In [ ]:
import src.models as models
reload(models)

# data params
model_inputs = ['H', 'V_mean']
model_outputs = ['H', 'V']
seq_length = 64
use_base_key = True
transpose = False
st = 0
nth_file = None

# network params
hierarchical = False
initial_state_from_dense = False
hidden_state = 512
lstm_layers = 2
dense_layers = 1
dense_size = 512
latent_size = 256
batch_size = 64
# ar_inputs only works as parameter for non hierarchical graph, currently
ar_inputs = None


model_input_reqs, model_output_reqs = models.get_model_reqs(model_inputs, model_outputs)


In [ ]:
z, model_inputs = models.create_LSTMencoder_graph(model_input_reqs, hidden_state_size=hidden_state, dense_size=dense_size, latent_size=latent_size, seq_length=seq_length)
encoder = tf.keras.Model(inputs=model_inputs, outputs=z, name=f'encoder')

z_input = tf.keras.Input(batch_shape=(1,)+z.shape[1:], name='z_in')

# must be stateful! With sequence length of 1.
pred, ar_inputs = models.create_LSTMdecoder_graph_ar(z_input, model_output_reqs, seq_length=1, hidden_state_size = hidden_state, dense_size=dense_size, stateful=True)

decoder = tf.keras.Model(inputs=[z_input] + ar_inputs, outputs=pred, name=f'decoder')

encoder.summary()
decoder.summary()

In [ ]:
run = 59
encoder.load_weights(f'experiments/run_{run}/{run}_best_train_weights.hdf5', by_name=True)
decoder.load_weights(f'experiments/run_{run}/{run}_best_train_weights.hdf5', by_name=True)

In [ ]:
# grab some examples, and predict z
idx = [0,45,70,100,125,150]
random_examples = {}

for md in model_datas.values():
    random_examples[md.name + '_in'] = md.data[idx,...]
    random_examples[md.name + '_in'] = random_examples[md.name + '_in']
    if md.seq:
        random_examples[md.name + '_ar'] = np.concatenate([np.zeros((len(idx),1, md.dim)), md.data[idx,...][:,:-1]], axis=-2)
z_pred = encoder.predict(random_examples)

In [ ]:
# predict on z
pred = []
inputs = {}
# get required ar inputs from model datas - these are just zeros
for out in model_output_reqs:
    if out.seq:
        inputs[out.name + '_ar'] = np.zeros((1,1,out.dim))
print([f'{name}: {md_in.shape}' for name, md_in in inputs.items()])

# What will unavailable for prediction?
non_accessible = ['V']

filter_ar_pos = True # whether to set all ar inputs to zero for positions with no notes 
for j, z in enumerate(z_pred):
    decoder.reset_states()
    # get latent vector
    inputs['z_in'] = np.expand_dims(z, 0)
    outputs_pred = []
    # iterate over timesteps
    for i in range(seq_length):
        # get the ar inputs
        for out in model_output_reqs:
            # will only be ar inputs if seq is true
            # in practice, not all seq inputs might be required by the model - but the model will select these by name if they are all passed
            if out.seq and out.name not in non_accessible:
                # need to expand dims - it is still seq data, just with time step of one
                inputs[out.name + '_ar'] = np.expand_dims(np.expand_dims(random_examples[out.name + '_ar'][j,i], 0), 0)
        decoded = decoder.predict(inputs)
        # iterate over outputs - feed put them into inputs
        # ignores whether the model actually needs them all autoregressively!
        for output_name, output in zip(decoder.output_names, decoded):
            input_name = output_name.split('_')[0] + '_ar'
            if filter_ar_pos:
                # set positions where there are no notes to zero
                # this assumes that all outputs have same dimensions as H! i.e., 88
                # this might error out, because of extra dimension on output
                output[:,:,np.where(random_examples['H_in'][j,i] != 1)[0]] = 0
                inputs[input_name] = np.expand_dims(np.expand_dims(output, 0), 0)
            # give outputs back autoregressively
#             print(input_name)
#             print(output)
            inputs[input_name] = output
        # need to add here ar input, but taken from the original H - this is known, so why use the model output!
#         inputs['H_ar'] = random_examples['H_ar'][j,i]
        outputs_pred.append(np.squeeze(decoded))
    pred.append(outputs_pred)

In [ ]:
# find axis that corresponds to velocity
v_index = np.where(np.array(decoder.output_names) == 'V_out')[0][0]
print('v_index:', v_index)
print('predictions shape:', np.array(pred).shape)
model_datas_pred = copy.deepcopy(model_datas)
model_datas_pred['V'].data[idx,...] = np.array(pred)[:,:,v_index,:]
for i in idx:
    pm_original = data.examples2pm(model_datas, i)
    pm_pred = data.examples2pm(model_datas_pred, i)
    pm_original.write(f'experiments/run_{run}/ex{i}original.mid')
    pm_pred.write(f'experiments/run_{run}/ex{i}prediction.mid')

### Hierarchical and/or Variational Prediction Attempt

In [ ]:
run = 216
client = pymongo.MongoClient()  # assuming a local MongoDB
fs = gridfs.GridFS(client.sacred)  # assuming database name is 'sacred'
runs = client.sacred.runs
# Now get run from the database
run_entry = runs.find_one({'_id': run})
config = run_entry['config']

In [ ]:
model_datas, seconds = data.folder2examples('training_data/midi_val', sparse=False, use_base_key=True, beats_per_ex=int(config['seq_length'] / 4), vel_cutoff=config['vel_cutoff'])
model_datas_pred = copy.deepcopy(model_datas)

random_examples, idx = data.n_rand_examples(model_datas)

#### predict using teacher forcing

In [ ]:
model_input_reqs, model_output_reqs = models.get_model_reqs(**config)
z, model_inputs = models.create_LSTMencoder_graph(model_input_reqs, **config)
if config['variational']:
    sampling_fn = models.sampling(**config)
    z = layers.Lambda(sampling_fn)(z)

# a few other 
config['stateful'] = False
config['ar_inc_batch_shape'] = False
# config['batch_size'] = 1

if config['variational']:
    sampling_fn = models.sampling(config['batch_size'], epsilon_std=config['epsilon_std'])
    # z_input is the tensor that will be passed into the decoder
    z_input = layers.Lambda(sampling_fn)(z)

else:
    z_input = z

if config['hierarchical']:
    build_decoder_graph = models.create_hierarchical_decoder_graph
else:
    build_decoder_graph =models.create_LSTMdecoder_graph_ar

if config['variational']:
pred, ar_inputs = models.build_decoder_graph(z, model_output_reqs, **config)
autoencoder = tf.keras.Model(inputs=model_inputs + ar_inputs, outputs=pred, name=f'autoencoder')

In [ ]:
models.load_weights_safe(autoencoder, f'experiments/run_{run}/{run}_best_val_weights.hdf5', by_name=False)

For non teacher forced prediction, there might be trouble with layer names. We can address this be saving weights from the autoencoder we just made - that newly saved version should have updated layer names. 

In [ ]:
pred = autoencoder.predict(random_examples)
# find axis that corresponds to velocity
v_index = np.where(np.array(autoencoder.output_names) == 'V_out')[0][0]
print('velocity index:', v_index)
model_datas_pred = copy.deepcopy(model_datas)
model_datas_pred['V'].data[idx,...] = np.array(pred)[v_index,:,:,:]
for i in idx:
    pm_original = data.examples2pm(model_datas, i)
    pm_pred = data.examples2pm(model_datas_pred, i)
    pm_original.write(f'experiments/run_{run}/ex{i}original.mid')
    pm_pred.write(f'experiments/run_{run}/ex{i}prediction_teacher_forced.mid')

#### predict the hard way

In [ ]:
model_input_reqs, model_output_reqs = models.get_model_reqs(**config)
z, model_inputs = models.create_LSTMencoder_graph(model_input_reqs, **config)
if config['variational']:
    sampling_fn = models.sampling(**config)
    z = layers.Lambda(sampling_fn)(z)

config['stateful'] = True
config['ar_inc_batch_shape'] = True
config['batch_size'] = 1
if config['hierarchical']:
    conductor_out, ar_inputs, decoder = models.create_hierarchical_decoder_graph(z, model_output_reqs, **config)
else:
    pred, ar_inputs = models.create_LSTMdecoder_graph_ar(z, model_output_reqs, **config)
autoconductor = tf.keras.Model(inputs=[model_inputs, ar_inputs], outputs=conductor_out, name=f'autoconductor')
# autoconductor.summary()

In [ ]:
# load some weights
models.load_weights_safe(autoconductor, f'experiments/run_{run}/{run}_best_train_weights.hdf5')
models.load_weights_safe(decoder, f'experiments/run_{run}/{run}_best_train_weights.hdf5')
# pred = autoencoder.predict(random_examples)

In [ ]:
auto_conductor_outs = autoconductor.predict(random_examples)

In [ ]:
conductor_substeps = int(config['seq_length'] / config['conductor_steps'])

# predict on z
pred = []
inputs = {}
# get required ar inputs from model datas - these are just zeros
for out in model_output_reqs:
    if out.seq:
        inputs[out.name + '_ar'] = np.zeros((1,1,out.dim))
print([f'{name}: {md_in.shape}' for name, md_in in inputs.items()])

# What will unavailable for prediction?
non_accessible = []

filter_ar_pos = False # whether to set all ar inputs to zero for positions with no notes 
for i, auto_out in enumerate(zip(*auto_conductor_outs)):
    print(f'Example {i + 1} of {len(idx)}')
    decoder.reset_states()
    outputs_pred = []
    # iterate over timesteps
    for c_step in range(config['conductor_steps']):
        for c_substep in range(conductor_substeps):
            t = c_step * conductor_substeps + c_substep
            if c_substep == 0:
                # get conductor output
                inputs['c_in'] = np.expand_dims(np.expand_dims(auto_out[0][c_step], 0), 0)
                # set initial states of decoder LSTMs
                for k in range(config['decoder_lstms']):
                    tf.keras.backend.set_value(decoder.get_layer(f'final_dec_LSTM_{k}').states[0], np.expand_dims(auto_out[k*2 + 1][c_step], 0))
                    tf.keras.backend.set_value(decoder.get_layer(f'final_dec_LSTM_{k}').states[1], np.expand_dims(auto_out[k*2 + 2][c_step], 0))
            # get the ar inputs
            for out in model_output_reqs:
                # will only be ar inputs if seq is true
                # in practice, not all seq inputs might be required by the model - but the model will select these by name if they are all passed
                if out.seq and out.name not in non_accessible:
                    # need to expand dims - it is still seq data, just with time step of one
                    inputs[out.name + '_ar'] = np.expand_dims(np.expand_dims(random_examples[out.name + '_ar'][i,t], 0), 0)
            decoded = decoder.predict(inputs)
            # iterate over outputs - feed put them into inputs
            # ignores whether the model actually needs them all autoregressively!
            for output_name, output in zip(decoder.output_names, decoded):
                input_name = output_name.split('_')[0] + '_ar'
                if filter_ar_pos:
                    # set positions where there are no notes to zero
                    # this assumes that all outputs have same dimensions as H! i.e., 88
                    output[...,np.where(random_examples['H_in'][i,t] != 1)[0]] = 0
                    inputs[input_name] = np.squeeze(output, 0)
                else:
                    inputs[input_name] = np.squeeze(output, 0)
            outputs_pred.append(np.squeeze(decoded))
    pred.append(outputs_pred)

In [ ]:
for t in range(32):
    print(min(np.array(pred)[0,t,v_index,:]), max(np.array(pred)[0,t,v_index,:]) - min(np.array(pred)[0,t,v_index,:]))

In [ ]:
# find axis that corresponds to velocity
v_index = np.where(np.array(decoder.output_names) == 'V_unconcat')[0][0]
print('v_index:', v_index)
print('predictions shape:', np.array(pred).shape)
model_datas_pred = copy.deepcopy(model_datas)
model_datas_pred['V'].data[idx,...] = np.array(pred)[:,:,v_index,:]
for i in idx:
    pm_original = data.examples2pm(model_datas, i)
    pm_pred = data.examples2pm(model_datas_pred, i)
    pm_original.write(f'experiments/run_{run}/ex{i}original.mid')
    pm_pred.write(f'experiments/run_{run}/run_{run}_ex{i}prediction.mid')

## Various mongodb operations
Code left over from doing a few things with the sacred mongodb.

In [ ]:
from bson.objectid import ObjectId
client = pymongo.MongoClient()  # assuming a local MongoDB
fs = gridfs.GridFS(client.sacred)  # assuming database name is 'sacred'

runs = client.sacred.runs
metrics = client.sacred.metrics
# Now get run from the database
run_entry = runs.find_one({'_id': 212})
metric_ids = {m['name']: ObjectId(m['id']) for m in run_entry['info']['metrics']}
# metrics_entry = metrics.find_one({'_id': metric_ids['V_out_categorical_crossentropy']})

# can always get weights like this, but need to write them to a temp file before loading
# weights = fs.get(run_entry['artifacts'][0]['file_id']).read()
print(run_entry.keys())
sizeof_fmt(sys.getsizeof(apply_backspaces_and_linefeeds(run_entry['captured_out'])))

In [ ]:
import json
import ast
for run_id in [140, 146, 171]:
    run_entry = runs.find_one({'_id': run_id})
    metric_ids = {m['name']: ObjectId(m['id']) for m in run_entry['info']['metrics']}

    epochs = run_entry['config']['epochs']
    with open(f'experiments/run_{run_id}/history-{epochs}epochs.json', 'rb') as f:
        hist = json.load(f)
        hist = ast.literal_eval(hist)

    metric_ids = {m['name']: ObjectId(m['id']) for m in run_entry['info']['metrics']}
    delta = (metrics_entry['timestamps'][-1] - metrics_entry['timestamps'][-2])
    timestamps = metrics_entry['timestamps']
    recorded_epochs = len(timestamps)
    # generate new timestamps
    timestamps_new = metrics_entry['timestamps'] + [timestamps[-1] + i * delta for i in range(1, epochs - recorded_epochs + 1)]



    for k, v in hist.items():
        metrics_entry = metrics.find_one({'_id': metric_ids[k]})
        if not metrics_entry is None:
            # get delta and new timesteps
            delta = (metrics_entry['timestamps'][-1] - metrics_entry['timestamps'][-2])
            timestamps = metrics_entry['timestamps']
            recorded_epochs = len(timestamps)
            # generate new timestamps
            timestamps_new = metrics_entry['timestamps'] + [timestamps[-1] + i * delta for i in range(1, epochs - recorded_epochs + 1)]
            agreement = sum([(a - b) < 0.00001 for a, b in zip(v[:10], metrics_entry['values'][:10])])
            if agreement == 10:
                metrics.update_one({'_id': metric_ids[k]}, {
                  '$set': {
                    'timestamps': timestamps_new, 'steps': [i for i in range(epochs)], 'values': v
                  }}
                    )
            else:
                print(f'problem with {k}')
                print('agreement:', agreement)


    
# metrics_entry['timestamps'][-1] + 2 * (metrics_entry['timestamps'][-1] - metrics_entry['timestamps'][-2])

In [ ]:
run_entries = runs.find({'_id': {'$lt': 216}})
for run in run_entries:
    print('processing run', run['_id'])
    c_out_filtered = '\n'.join([s for s in apply_backspaces_and_linefeeds(run['captured_out']).split('\n') if '>.' not in s and '....' not in s and s != ''])
    runs.update_one({'_id': run['_id']}, {
      '$set': {
        'captured_out': c_out_filtered
      }}
        )


In [ ]:
run_entries = runs.find({'_id': {'$lt': 221, '$gt': 216}})
for run in run_entries:
    print(run['_id'])
    print(run['captured_out'][:120])
    print(sizeof_fmt(sys.getsizeof(run['captured_out'])))

In [ ]:
run_entries = runs.find({'_id': {'$lt': 221, '$gt': 216}})
for run in run_entries:
    print(run['_id'])
    print(run['captured_out'][:120])
    print(sizeof_fmt(sys.getsizeof(run['captured_out'])))

In [ ]:
run_entry = runs.find_one({'_id': {'$lt': 221}})
# use set for setting fields. unset removes fields.
# This example finds runs with id <= 146, with field config.z_activation, and updates the field to relu
runs.update_many({'_id': {'$lte': 146}, 'config.z_activation': {'$exists': True}}, {
  '$set': {
    'config.z_activation': 'relu'
  }}
    )
# print(runs.find_one({'_id': 146})['config']['z_activation'])